<a href="https://colab.research.google.com/github/tatendakasirori/AI_4_ALL_project/blob/main/src/viirs/viirs_data_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data collection
- Collected ALAN, snow flag, cloud mask, and luner rediance values for all the data in the wanted range of dates.
- Also collected the each single night’s global composite, created by combining multiple VIIRS DNB observations taken over that night.
- For each state, VT and NJ, there is a folder contating all the need images and a single csv file.
- The images are named after the date they correspond to and are .tiff.
- The CSV file has the name of every image for its state, and the following data corresponding to it: DNB_BRDF_Corrected_NTL, QF_Cloud_Mask, Snow_Flag, DNB_Lunar_Irradiance.

In [ ]:
import os
import os
import pandas as pd
import geemap
import ee

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='hypnotic-trees-453820-e5')


# --- Settings ---
ranges = [
      ("2021-03-01", "2021-06-15"),
      ("2022-08-01", "2022-11-15"),
      ("2023-03-01", "2023-06-15"),
      ("2023-08-01", "2023-11-15"),
      ("2024-03-01", "2024-06-15"),
      ("2024-08-01", "2024-11-15"),
      ("2025-03-01", "2025-06-15"),
      ("2025-08-01", "2025-11-04"),
  ]

# Output directories
base_dir = "/content/viirs_data"
vt_dir, nj_dir = [os.path.join(base_dir, s) for s in ["VT", "NJ"]]
os.makedirs(vt_dir, exist_ok=True)
os.makedirs(nj_dir, exist_ok=True)

# --- VIIRS collection with multiple bands ---
bands = [
    "DNB_BRDF_Corrected_NTL",
    "DNB_Lunar_Irradiance",
    "Gap_Filled_DNB_BRDF_Corrected_NTL",
    "Latest_High_Quality_Retrieval",
    "Mandatory_Quality_Flag",
    "QF_Cloud_Mask",
    "Snow_Flag",
]

viirs = ee.ImageCollection("NASA/VIIRS/002/VNP46A2").select(bands)

# --- State boundaries ---
vt = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq("STUSPS", "VT"))
nj = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq("STUSPS", "NJ"))

# --- Function to get daily means ---
def get_viirs_stats(img, region):
    """Compute mean values for selected bands over a given region."""
    mean_dict = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region.geometry(),
        scale=500,
        maxPixels=1e13
    )
    return mean_dict

# --- Download function ---
def download_viirs_images(state_fc, state_code, out_dir):
    records = []

    for start, end in ranges:
        ic = viirs.filterDate(start, end)
        date_list = ee.List(ic.aggregate_array("system:time_start")).map(lambda d: ee.Date(d).format("YYYY-MM-dd")).getInfo()

        for d in date_list:
            print(f"Processing {state_code} - {d} ...")
            img = ic.filterDate(d, ee.Date(d).advance(1, "day")).mean().clip(state_fc)

            # Export image (all bands)
            name = f"{state_code}_{d}.tif"
            path = os.path.join(out_dir, name)
            geemap.ee_export_image(
                img,
                filename=path,
                scale=500,
                region=state_fc.geometry(),
                file_per_band=False
            )

            # Compute mean stats
            stats = get_viirs_stats(img, state_fc).getInfo()
            stats["date"] = d
            stats["filename"] = name
            records.append(stats)

    # Save stats for all images in this state
    df = pd.DataFrame(records)
    csv_path = os.path.join(out_dir, f"{state_code}_metadata.csv")
    df.to_csv(csv_path, index=False)
    print(f"Saved metadata to {csv_path}")

# --- Run downloads ---
download_viirs_images(vt, "VT", vt_dir)
download_viirs_images(nj, "NJ", nj_dir)

Streaming output truncated to the last 5000 lines.
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2025-03-20.tif
Processing VT - 2025-03-21 ...
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2025-03-21.tif
Processing VT - 2025-03-22 ...
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2025-03-22.tif
Processing VT - 2025-03-23 ...
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2025-03-23.tif
Processing VT - 2025-03-24 ...
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2025-03-24.tif
Processing VT - 2025-03-25 ...
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2025-03-25.tif
Processing VT - 2025-03-26 ...
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2025-03-26.tif
Processing VT - 2025-03-27 ...
Generating URL ...
Please wait ...
Data downloaded to /content/viirs_data/VT/VT_2025-0

# Downloading The Data
- The file is first zipped because it is large

In [ ]:
!zip -r viirs_data.zip /content/viirs_data
from google.colab import files
files.download("viirs_data.zip")


  adding: content/viirs_data/ (stored 0%)
  adding: content/viirs_data/NJ/ (stored 0%)
  adding: content/viirs_data/NJ/NJ_2025-10-03.tif (deflated 11%)
  adding: content/viirs_data/NJ/NJ_2023-10-23.tif (deflated 10%)
  adding: content/viirs_data/NJ/NJ_2022-09-10.tif (deflated 10%)
  adding: content/viirs_data/NJ/NJ_2024-11-01.tif (deflated 4%)
  adding: content/viirs_data/NJ/NJ_2024-09-04.tif (deflated 12%)
  adding: content/viirs_data/NJ/NJ_2025-03-17.tif (deflated 2%)
  adding: content/viirs_data/NJ/NJ_2025-03-09.tif (deflated 9%)
  adding: content/viirs_data/NJ/NJ_2025-06-07.tif (deflated 2%)
  adding: content/viirs_data/NJ/NJ_2025-05-30.tif (deflated 2%)
  adding: content/viirs_data/NJ/NJ_2023-05-24.tif (deflated 12%)
  adding: content/viirs_data/NJ/NJ_2024-11-08.tif (deflated 11%)
  adding: content/viirs_data/NJ/NJ_2022-09-08.tif (deflated 5%)
  adding: content/viirs_data/NJ/NJ_2025-10-02.tif (deflated 10%)
  adding: content/viirs_data/NJ/NJ_2025-10-16.tif (deflated 10%)
  adding:

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>